Exemplo abrindo precipitaçao com Google Earth Engine e plotando para uma posição a precipitação
Para rodar será necessário fazer uma cópia no seu Google Drive. Menu File -> Save copy in Drive.

In [ ]:
# instalando geemap atualizado (https://geemap.org/)
# se estiver no jupyterlab rodar:
%pip install -U "geemap[workshop]" -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.6 MB/s eta 0:00:00


In [85]:
import ee
import geemap
import numpy as np
import datetime
import pandas as pd
import plotly.express as px

In [86]:
# vai pedir a autentificação (tem que ter conta gmail)
m = geemap.Map()

In [ ]:
# pegando variavel precipitação
# para as demais ver: https://gee-community-catalog.org/projects/br_dwgd/
var = ee.ImageCollection("projects/sat-io/open-datasets/BR-DWGD/PR")

# para reescalonar
offset = 225
scale  = 0.006866665

# reescalonando
def scaleBand(image):
    scaledImage = image.select("b1").rename(['pr']).multiply(scale).add(offset)
    return scaledImage.copyProperties(image, image.propertyNames())

var = var.map(scaleBand)
color_pal = ['000066', '001199', '0044BB', '0077DD', '33AAEE', '66CCFF', 'FFDDCC', 'FFBB99', 'FF9966', 'FF6644']
vis = {'min': 0,'max': 70, 'palette': color_pal}

# definindo período (tem limite)
start_date, end_date = '2010-01-01','2023-01-01'
var = var.filterDate(start_date, end_date).select(['pr']);
m.addLayer(var.first(), vis, 'Precipitação')

In [ ]:
# plotando a precipitação da data inicial
m.centerObject(var.first())
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Pegando infromações para uma posição

In [ ]:
# coordenada do ponto de interesse [lon, lat]
lon_lat = [-43.04, -14.33]
point = {'type':'Point', 'coordinates': lon_lat}
info = var.getRegion(point, 500).getInfo()

Plotando dados diários para o período

In [ ]:
header = info[0]
data = np.array(info[1:])
iTime = header.index('time')
time = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:,iTime].astype(int))]
df = pd.DataFrame(data[:,-1].astype('float'), columns=['Precipitação (mm)'])
df.set_index(np.array(time), inplace=True)

# plotando
fig = px.scatter(df, x=df.index, y='Precipitação (mm)',
                 title=f"Precipitação na lat={lon_lat[1]}; lon={lon_lat[0]}")
fig.show()

Para salvar em formato "csv"

In [ ]:
# df.to_csv('dados.csv')
df

,Precipitação (mm)
2010-01-01,4.209254
2010-01-02,6.090720
2010-01-03,-0.000012
2010-01-04,-0.000012
2010-01-05,-0.000012
...,...
2022-12-27,0.363921
2022-12-28,0.096121
2022-12-29,0.034321
2022-12-30,-0.000012


from matplotlib import pyplot as plt
df['Precipitação (mm)'].plot(kind='hist', bins=20, title='Precipitação (mm)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['Precipitação (mm)'].plot(kind='line', figsize=(8, 4), title='Precipitação (mm)')
plt.gca().spines[['top', 'right']].set_visible(False)